In [4]:
import pandas as pd
import numpy as np
import pyreadr
import math

In [5]:
data_read = pyreadr.read_r("../data/wage2015_subsample_inference.Rdata")
data = data_read[ 'data' ]
data.shape

(5150, 20)

In [6]:
data

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
rownames,,,,,,,,,,,,,,,,,,,,
10,9.615385,2.263364,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600,11,8370,18
12,48.076923,3.872802,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050,10,5070,9
15,11.057692,2.403126,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260,19,770,4
18,13.942308,2.634928,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420,1,6990,12
19,28.846154,3.361977,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015,6,9470,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32620,14.769231,2.692546,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,9.0,0.81,0.729,0.6561,4700,16,4970,9
32624,23.076923,3.138833,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,12.0,1.44,1.728,2.0736,4110,13,8680,20
32626,38.461538,3.649659,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,11.0,1.21,1.331,1.4641,1550,4,3680,6


In [7]:

m = data[ ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1",'exp2',"exp3","exp4",'occ', 'occ2','ind','ind2'] ]

college=m[m['clg']==1]
m_college= college[ ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1",'exp2',"exp3","exp4",'occ', 'occ2','ind','ind2'] ]
scl=m[m['scl']==1]
m_scl=scl[["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1",'exp2',"exp3","exp4",'occ', 'occ2','ind','ind2']]

In [8]:
m_college.shape

(1636, 19)

In [9]:
m_scl.shape

(1432, 19)

In [10]:
total=pd.concat([m_college,m_scl])
total.shape

(3068, 19)

In [11]:
total.reset_index(inplace=True)

In [12]:
total2 = total[ ["lwage","sex","scl","clg","ne","mw","so","we","exp1",'exp2',"exp3","exp4",'occ', 'occ2','ind','ind2']]

total3 = total2[ ["lwage","sex","scl","clg","ne","mw","so","we","exp1"]]

data_female = total3[total3[ 'sex' ] == 1 ]
t_female = data_female[ ["lwage","sex","scl","clg","ne","mw","so","we","exp1"] ]

data_male = total3[ total3[ 'sex' ] == 0 ]
t_male = data_male[ [ "lwage","sex","scl","clg","ne","mw","so","we","exp1" ] ]


table = np.zeros( (9, 3) )
table[:, 0] = total3.mean().values
table[:, 1] = t_male.mean().values
table[:, 2] = t_female.mean().values
table_pandas = pd.DataFrame( table, columns = [ 'All', 'Men', 'Women'])
table_pandas.index = ["Log Wage","Sex","Some College","Gollage Graduate","Northeast","Midwest","South","West","Experience"]
table_html = table_pandas.to_html()

table_pandas

,All,Men,Women
Log Wage,3.000022,3.038412,2.956904
Sex,0.470991,0.000000,1.000000
Some College,0.466754,0.481824,0.449827
Gollage Graduate,0.533246,0.518176,0.550173
Northeast,0.226532,0.219347,0.234602
Midwest,0.265971,0.261245,0.271280
South,0.285854,0.290819,0.280277
West,0.221643,0.228589,0.213841
Experience,12.700945,12.433148,13.001730


In [13]:
data_female['lwage'].mean() - data_male['lwage'].mean()

-0.08150855508735866

In [14]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [15]:
nocontrol_model = smf.ols( formula = 'lwage ~ sex', data = total2 )
nocontrol_est = nocontrol_model.fit().summary2().tables[1]['Coef.']['sex']
HCV_coefs = nocontrol_model.fit().cov_HC0
nocontrol_se = np.power( HCV_coefs.diagonal() , 0.5)[1]

In [16]:
print( f'The estimated gender coefficient is {nocontrol_est} and the corresponding robust standard error is {nocontrol_se}' )

The estimated gender coefficient is -0.08150855508736156 and the corresponding robust standard error is 0.019579647767772403


# OLS

In [17]:
#["lwage","sex","scl","clg","ne","mw","so","we","exp1"]
flex = 'lwage ~ sex + (exp1+exp2+exp3+exp4)*(clg+mw+so+we+occ2+ind2)'
control_model = smf.ols( formula = flex, data = total2 )
control_est = control_model.fit().summary2().tables[1]['Coef.']['sex']

print(control_model.fit().summary2().tables[1])
print(f"Coefficient for OLS with controls {control_est}" )

HCV_coefs = control_model.fit().cov_HC0
control_se = np.power( HCV_coefs.diagonal() , 0.5)[1]

               Coef.  Std.Err.         t         P>|t|    [0.025    0.975]
Intercept   2.985101  0.336482  8.871492  1.250129e-18  2.325327  3.644876
occ2[T.10]  0.091982  0.243220  0.378184  7.053225e-01 -0.384925  0.568888
occ2[T.11] -0.499418  0.436858 -1.143202  2.530511e-01 -1.356010  0.357175
occ2[T.12]  0.190101  0.341142  0.557249  5.774012e-01 -0.478810  0.859012
occ2[T.13] -0.194529  0.271881 -0.715492  4.743637e-01 -0.727633  0.338575
...              ...       ...       ...           ...       ...       ...
exp3:we    -0.230864  0.184398 -1.251987  2.106777e-01 -0.592431  0.130704
exp4:clg   -0.013467  0.020134 -0.668849  5.036463e-01 -0.052945  0.026012
exp4:mw     0.014287  0.025814  0.553477  5.799802e-01 -0.036328  0.064902
exp4:so    -0.003759  0.022547 -0.166725  8.675981e-01 -0.047968  0.040450
exp4:we     0.028286  0.023812  1.187890  2.349761e-01 -0.018405  0.074978

[231 rows x 6 columns]
Coefficient for OLS with controls -0.053062340357757656


In [19]:
control_se

0.19319010784693572

## Partialling-Out using ols

In [20]:
# models
# model for Y
flex_y = 'lwage ~  (exp1+exp2+exp3+exp4)*(clg+mw+so+we+occ2+ind2)'
# model for D
flex_d = 'sex ~ (exp1+exp2+exp3+exp4)*(clg+mw+so+we+occ2+ind2)' 

# partialling-out the linear effect of W from Y
t_Y = smf.ols( formula = flex_y , data = total2 ).fit().resid

# partialling-out the linear effect of W from D
t_D = smf.ols( formula = flex_d , data = total2 ).fit().resid

data_res = pd.DataFrame( np.vstack(( t_Y.values , t_D.values )).T , columns = [ 't_Y', 't_D' ] )
# regression of Y on D after partialling-out the effect of W
partial_fit =  smf.ols( formula = 't_Y ~ t_D' , data = data_res ).fit()
partial_est = partial_fit.summary2().tables[1]['Coef.']['t_D']

print("Coefficient for D via partialling-out", partial_est)

# standard error
HCV_coefs = partial_fit.cov_HC0
partial_se = np.power( HCV_coefs.diagonal() , 0.5)[1]

# confidence interval
partial_fit.conf_int( alpha=0.05 ).iloc[1, :]

Coefficient for D via partialling-out -0.053062340357753764


0   -0.089571
1   -0.016554
Name: t_D, dtype: float64

In [21]:
yhat=list(smf.ols( formula = flex_y , data = total2 ).fit().predict())

In [22]:
total2['ypredicho']=yhat

<ipython-input-22-b9d16deb59ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total2['ypredicho']=yhat


In [23]:
total2

,lwage,sex,scl,clg,ne,mw,so,we,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2,ypredicho
0,2.263364,1.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,0.4900,0.343000,0.240100,3600,11,8370,18,2.762284
1,3.872802,0.0,0.0,1.0,1.0,0.0,0.0,0.0,31.0,9.6100,29.791000,92.352100,3050,10,5070,9,3.575963
2,3.361977,1.0,0.0,1.0,1.0,0.0,0.0,0.0,22.0,4.8400,10.648000,23.425600,2015,6,9470,22,3.222386
3,2.462215,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0100,0.001000,0.000100,1650,5,7460,14,2.938010
4,2.956512,1.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.1600,0.064000,0.025600,3255,10,8190,18,3.143722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3063,2.486508,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,0.1600,0.064000,0.025600,9620,22,6170,10,2.578097
3064,3.649659,1.0,1.0,0.0,0.0,0.0,0.0,1.0,7.5,0.5625,0.421875,0.316406,220,1,770,4,2.995180
3065,3.328075,0.0,1.0,0.0,0.0,0.0,0.0,1.0,28.5,8.1225,23.149125,65.975006,8140,21,770,4,3.033369
3066,2.486508,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.5,9.9225,31.255875,98.456006,5510,17,6380,10,2.901543


In [24]:
import matplotlib.pyplot as plt

In [25]:
total5=total2[total2['clg']==1]

In [26]:
total6=total2[total2['scl']==1]